<a href="https://colab.research.google.com/github/stachibana/google-colab-linebot/blob/master/LINEBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# このノートブックをインターネットに公開

In [ ]:
!ssh-keygen -q -t rsa -N '' -f ~/.ssh/id_rsa <<<y >/dev/null 2>&1
!nohup ssh -R 80:localhost:8000 -o StrictHostKeyChecking=no localhost.run &> /dev/null &
!sleep 5
!echo 'あなたのWebhook URLはこちらです。'
!cat nohup.out | egrep -o 'https?://[^ ]+' | tail -1

# LINE Botの準備

[LINE Developers コンソール](https://developers.line.biz/ja/)からMessaging APIの新規チャネルを作成し、Webhook URLに上記URLを入力してください。

詳しくは、
* [LINE Developersコンソールでチャネルを作成する](https://developers.line.biz/ja/docs/messaging-api/getting-started/#using-console)
* [ボットを作成する](https://developers.line.biz/ja/docs/messaging-api/building-bot/)

を参照してください。

Webhook URLには前セクションで作成された https://XXXXXXXXXXXXXXX.localhost.run/ の部分を入力してください。

# Channel Secret、Channel Access Tokenの設定

以下のChannel Secret、Channel Access TokenをLINE Developers コンソールから取得したものに置き換えてください。


In [3]:
CHANNEL_SECRET = "YOUR_CHANNEL_SECRET"
CHANNEL_ACCESS_TOKEN = "YOUR_CHANNEL_ACCESS_TOKEN"

# パッケージのインストール

In [4]:
!pip install flask line-bot-sdk -q

# サービスを起動

In [ ]:
from flask import Flask, request, abort

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)

app = Flask(__name__)

line_bot_api = LineBotApi(CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(CHANNEL_SECRET)

@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=event.message.text))


if __name__ == "__main__":
    app.run(port=8000)

# コードの編集

コードを編集したいときは実行中のサービスを停止して編集、再度実行してください。トンネルの再実行やWebhookの書き換えは必要ありません。

なお、このノートブックのランタイムは非操作状態で90分、最大12時間後に破棄されます。その際は開き直し、Webhookの書き換えが必要になります。